##SETUP

In [40]:
import os
import time
import timeit
import pandas as pd
import random
import numpy as np
# Android ADB
from scrcpy import Client, const
from subprocess import check_output,Popen
import threading

# Image processing
from PIL import Image
import cv2
import pytesseract
# Notebook
from IPython.display import display,clear_output
from tqdm.notebook import trange, tqdm

import sys
# Start Scrcpy once per restart
if 'importlib' not in sys.modules:  
    start_scrcpy=True
    if start_scrcpy:
        proc = Popen(['C:\Programs\Scrcpy\scrcpy'], shell=True)
        time.sleep(6) # <-- sleep for 6''
        proc.terminate() # <-- terminate the process
# internal
import bot_core
import bot_perception

import importlib
importlib.reload(bot_core)
importlib.reload(bot_perception)


SLEEP_DELAY=0.1


bot = bot_core.Bot()

In [28]:
def refresh_shop(self):
    self.click_button((100,1500)) # Click store button
    [self.swipe([0,0],[2,0]) for i in range(20)] # swipe to top
    self.swipe([2,0],[0,0]) # Swipe down once
    time.sleep(1)
    self.click(30,150) # stop scroll
    avail_buttons = self.get_current_icons(available=True)
    if (avail_buttons == 'refresh_button.png').any(axis=None):
        pos = bot_core.get_button_pos(avail_buttons,'refresh_button.png')
        # Buy first and last item (possible legendary) before refresh
        #self.click_button(pos-[300,820]) # Click first (free) item
        #self.click(400,1150) # buy
        #self.click(30,150) # remove pop-up
        #self.click_button(pos+[400,-400]) # Click first (free) item
        #self.click(400,1150) # buy
        print('Bought!')
        self.click_button(pos)
        print('refreshed!')
    return avail_buttons

def watch_ads(self):
    avail_buttons = self.get_current_icons(available=True)
    # Watch ad if available
    if (avail_buttons == 'quest_done.png').any(axis=None):
        pos = bot_core.get_button_pos(avail_buttons,'quest_done.png')
        self.click_button(pos)
        self.click(700,600) # collect second completed quest
        self.click(700,400) # collect second completed quest
        [self.click(150,250) for i in range(2)] # click dailies twice
        self.click(420,420) # collect ad chest
    elif (avail_buttons == 'ad_season.png').any(axis=None):
        pos = bot_core.get_button_pos(avail_buttons,'ad_season.png')
        self.click_button(pos)
    elif (avail_buttons == 'ad_pve.png').any(axis=None):
        pos = bot_core.get_button_pos(avail_buttons,'ad_pve.png')
        self.click_button(pos)
    elif (avail_buttons == 'store_refresh.png').any(axis=None):
        refresh_shop(self)
    elif (avail_buttons == 'refresh_button.png').any(axis=None):
        refresh_shop(self,buy=True)
    else:
        print('Watched all ads!')
        return
    # Keep watching until back in menu
    for i in range(20):
        avail_buttons,status = self.battle_screen()
        if status =='menu':
            print('FINISHED AD')
            return
        time.sleep(2)
        self.click(870,30) # skip forward/click X
        self.click(870,100) # click X playstore popup
        self.shell(f'input keyevent {const.KEYCODE_BACK}') #Force back
        print('AD TIME',i,status)
    # Restart game if can't escape ad
    self.restart_RR()
watch_ads(bot)

Watched all ads!


## RUN BOT

In [41]:
wait=0
combat = 0
grid_df =None
watch_ad = True
# Main loop
while(1):
    output = bot.battle_screen(True,True,floor=5)
    if output[1]=='fighting':
        watch_ad = True 
        wait = 0
        combat+=1
        if combat>50:
            bot.restart_RR()
            combat = 0
            continue
        battle_bar=trange(8)
        for i in battle_bar:
                time.sleep(0.2)
                # Spawn units
                bot.click(450,1360)
                # Upgrade units
                bot.mana_level([3,1,5,4],hero_power=True)
                # Try to merge units
                grid_df,unit_series,merge_series,df_groups,info = bot.try_merge(prev_grid=grid_df)
                clear_output()
                display([f'{combat}, {i+1}/8 {output[1]}, {info}', 'Average age: '+ str(grid_df['Age'].mean().round(2)),grid_df,merge_series,df_groups])
    else:
        if watch_ad:
            for i in range(5):
                watch_ads(bot)
            watch_ad = False
        combat=0
        wait+=1
        if wait>40:
            print('RESTARTING')
            bot.restart_RR()
            wait=0
        print(output[1],'wait count:',wait)

['1, 1/8 fighting, need more units!',
 'Average age: 3.53',
     grid_id        unit  probability  rank  rank_error position  Age
 0    icon_0   empty.png        999.0     0         0.0   [0, 0]  4.0
 1    icon_1   empty.png        999.0     0         0.0   [0, 1]  5.0
 2    icon_2   empty.png        700.0     0         0.0   [0, 2]  5.0
 3    icon_3  hunter.png        440.0     1     11860.0   [0, 3]  0.0
 4    icon_4   empty.png        999.0     0         0.0   [0, 4]  5.0
 5    icon_5   empty.png        999.0     0         0.0   [1, 0]  1.0
 6    icon_6   empty.png        999.0     0         0.0   [1, 1]  1.0
 7    icon_7   empty.png        999.0     0         0.0   [1, 2]  1.0
 8    icon_8   empty.png        999.0     0         0.0   [1, 3]  5.0
 9    icon_9   empty.png        999.0     0         0.0   [1, 4]  1.0
 10  icon_10   empty.png        999.0     0         0.0   [2, 0]  5.0
 11  icon_11   empty.png        999.0     0         0.0   [2, 1]  5.0
 12  icon_12   empty.png      

# Play Game

In [2]:
# Restart game
bot.restart_RR()

## Quick restart

In [20]:
# Drop co-op game
for i in range(30):
    bot.restart_RR(True)